# 레시피 20 : Method Chaining 소개

In [12]:
# 간단한 메서드 체이닝 예시
import polars as pl
import seaborn as sns

# iris 데이터셋 로드
df = pl.from_pandas(sns.load_dataset('iris'))

# select와 sort를 체이닝으로 연결
result = (df
    .select(['sepal_length', 'sepal_width', 'species'])
    .sort('sepal_length', descending=True)
)

print(result.head(3))

shape: (3, 3)
┌──────────────┬─────────────┬───────────┐
│ sepal_length ┆ sepal_width ┆ species   │
│ ---          ┆ ---         ┆ ---       │
│ f64          ┆ f64         ┆ str       │
╞══════════════╪═════════════╪═══════════╡
│ 7.9          ┆ 3.8         ┆ virginica │
│ 7.7          ┆ 3.8         ┆ virginica │
│ 7.7          ┆ 2.6         ┆ virginica │
└──────────────┴─────────────┴───────────┘


# 레시피 21 : 데이터 파이프라인

In [4]:
# 1. 기본적인 데이터 전처리 파이프라인
df_processed = (df
    .select(['sepal_length', 'sepal_width', 'petal_length', 'species'])  # 필요한 컬럼만 선택
    .filter(pl.col('sepal_length') > 5.0)  # sepal_length가 5.0보다 큰 행만 필터링
    .with_columns([
        pl.col('sepal_length').mean().alias('avg_sepal_length'),  # 평균 sepal_length 컬럼 추가
        pl.col('petal_length').mean().alias('avg_petal_length')   # 평균 petal_length 컬럼 추가
    ])
)

print("기본 전처리된 데이터:")
print(df_processed.head(3))

기본 전처리된 데이터:
shape: (3, 6)
┌──────────────┬─────────────┬──────────────┬─────────┬──────────────────┬──────────────────┐
│ sepal_length ┆ sepal_width ┆ petal_length ┆ species ┆ avg_sepal_length ┆ avg_petal_length │
│ ---          ┆ ---         ┆ ---          ┆ ---     ┆ ---              ┆ ---              │
│ f64          ┆ f64         ┆ f64          ┆ str     ┆ f64              ┆ f64              │
╞══════════════╪═════════════╪══════════════╪═════════╪══════════════════╪══════════════════╡
│ 5.1          ┆ 3.5         ┆ 1.4          ┆ setosa  ┆ 6.129661         ┆ 4.315254         │
│ 5.4          ┆ 3.9         ┆ 1.7          ┆ setosa  ┆ 6.129661         ┆ 4.315254         │
│ 5.4          ┆ 3.7         ┆ 1.5          ┆ setosa  ┆ 6.129661         ┆ 4.315254         │
└──────────────┴─────────────┴──────────────┴─────────┴──────────────────┴──────────────────┘


# 레시피 22 : 집계 파이프라인

In [9]:
# 2. 집계 파이프라인
df_aggregated = (df
    .group_by('species')
    .agg([
        pl.col('sepal_length').mean().alias('avg_sepal_length'),
        pl.col('petal_length').mean().alias('avg_petal_length'),
        pl.col('sepal_width').std().alias('std_sepal_width')
    ])
    .sort('avg_sepal_length', descending=True)
)

print("\n종별 집계 데이터:")
print(df_aggregated)


종별 집계 데이터:
shape: (3, 4)
┌────────────┬──────────────────┬──────────────────┬─────────────────┐
│ species    ┆ avg_sepal_length ┆ avg_petal_length ┆ std_sepal_width │
│ ---        ┆ ---              ┆ ---              ┆ ---             │
│ str        ┆ f64              ┆ f64              ┆ f64             │
╞════════════╪══════════════════╪══════════════════╪═════════════════╡
│ virginica  ┆ 6.588            ┆ 5.552            ┆ 0.322497        │
│ versicolor ┆ 5.936            ┆ 4.26             ┆ 0.313798        │
│ setosa     ┆ 5.006            ┆ 1.462            ┆ 0.379064        │
└────────────┴──────────────────┴──────────────────┴─────────────────┘


In [8]:
%%time
df = pl.from_pandas(sns.load_dataset('iris'))

# select, filter, sort를 독립적으로 사용
df = df.select(['species', 'petal_length', 'petal_width'])

# 그 다음 petal_length가 4.0보다 큰 행만 필터링
df = df.filter(pl.col('petal_length') > 4.0)

# 마지막으로 petal_width를 기준으로 내림차순 정렬
df = df.sort('petal_width', descending=True)

print("필터링 및 정렬된 iris 데이터:")
print(df.head(5))

필터링 및 정렬된 iris 데이터:
shape: (5, 3)
┌───────────┬──────────────┬─────────────┐
│ species   ┆ petal_length ┆ petal_width │
│ ---       ┆ ---          ┆ ---         │
│ str       ┆ f64          ┆ f64         │
╞═══════════╪══════════════╪═════════════╡
│ virginica ┆ 6.0          ┆ 2.5         │
│ virginica ┆ 6.1          ┆ 2.5         │
│ virginica ┆ 5.7          ┆ 2.5         │
│ virginica ┆ 5.1          ┆ 2.4         │
│ virginica ┆ 5.6          ┆ 2.4         │
└───────────┴──────────────┴─────────────┘
CPU times: total: 0 ns
Wall time: 9.02 ms


In [9]:
%%time
df = pl.from_pandas(sns.load_dataset('iris'))

# select, filter, sort를 Method Chaining으로 사용
df = (df
    .select(['species', 'petal_length', 'petal_width'])
    .filter(pl.col('petal_length') > 4.0)
    .sort('petal_width', descending=True)
)

print("필터링 및 정렬된 iris 데이터:")
print(df.head(5))

필터링 및 정렬된 iris 데이터:
shape: (5, 3)
┌───────────┬──────────────┬─────────────┐
│ species   ┆ petal_length ┆ petal_width │
│ ---       ┆ ---          ┆ ---         │
│ str       ┆ f64          ┆ f64         │
╞═══════════╪══════════════╪═════════════╡
│ virginica ┆ 6.0          ┆ 2.5         │
│ virginica ┆ 6.1          ┆ 2.5         │
│ virginica ┆ 5.7          ┆ 2.5         │
│ virginica ┆ 5.1          ┆ 2.4         │
│ virginica ┆ 5.6          ┆ 2.4         │
└───────────┴──────────────┴─────────────┘
CPU times: total: 0 ns
Wall time: 5.48 ms


# 레시피 23 : 조건부 파이프라인

In [11]:
# iris 데이터셋을 Polars DataFrame으로 변환
df = pl.from_pandas(sns.load_dataset('iris'))

# 조건부 파이프라인 구성
df_conditional = (df
    # with_columns: 새로운 컬럼을 추가
    .with_columns([
        # when().then().otherwise(): SQL의 CASE WHEN과 유사한 조건부 로직
        pl.when(pl.col('sepal_length') > 6.0)  # 조건: sepal_length가 6.0보다 큰 경우
        .then(pl.lit('large'))  # 조건이 참일 때 'large' 값 할당
        .otherwise(pl.lit('small'))  # 조건이 거짓일 때 'small' 값 할당
        .alias('size_category')  # 새로운 컬럼의 이름을 'size_category'로 지정
    ])
    # group_by: 지정된 컬럼들로 그룹화
    .group_by(['species', 'size_category'])
    # agg: 그룹별 집계 연산 수행
    .agg(
        cnt=pl.len()  # 각 그룹의 행 개수를 계산
    )
    # sort: 지정된 컬럼들로 정렬
    .sort(['species', 'size_category'])
)

print("\n조건부 분류 결과:")
print(df_conditional)


조건부 분류 결과:
shape: (5, 3)
┌────────────┬───────────────┬─────┐
│ species    ┆ size_category ┆ cnt │
│ ---        ┆ ---           ┆ --- │
│ str        ┆ str           ┆ u32 │
╞════════════╪═══════════════╪═════╡
│ setosa     ┆ small         ┆ 50  │
│ versicolor ┆ large         ┆ 20  │
│ versicolor ┆ small         ┆ 30  │
│ virginica  ┆ large         ┆ 41  │
│ virginica  ┆ small         ┆ 9   │
└────────────┴───────────────┴─────┘
